In [ ]:
import torch, os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import torchvision as vision
import cv2

%matplotlib inline
torch.__version__

In [ ]:
!git clone https://github.com/leoxiaobin/deep-high-resolution-net.pytorch
HRNET_DIR =  '/content/deep-high-resolution-net.pytorch'
if os.getcwd()!=HRNET_DIR:
  os.chdir(HRNET_DIR)
!pip install -r requirements.txt

In [ ]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
from pycocotools.coco import COCO

In [ ]:
HRNET_DRIVE_DIR = '/content/drive/My Drive/Envisat/HRNet_regression/'


if os.getcwd()!=HRNET_DRIVE_DIR:
  os.chdir(HRNET_DRIVE_DIR)

In [ ]:
def compute_bbox(img_size, uv_pt, margin = 5):
        '''


        Parameters
        ----------
        uv_pt : TYPE
            DESCRIPTION.
        margin : TYPE, optional
            DESCRIPTION. The default is 5.

        Returns
        -------
        list
            DESCRIPTION.

        '''
        #margin:percent : default  5% relaxed in w/h
        relax_margin_u= (margin/100)*(max(uv_pt[0])-min(uv_pt[0]))
        relax_margin_v= (margin/100)*(max(uv_pt[1])-min(uv_pt[1]))
        u_max = max(uv_pt[0])+relax_margin_u if max(uv_pt[0])+relax_margin_u <=img_size[0] else img_size[0]
        u_min = min(uv_pt[0])-relax_margin_u if min(uv_pt[0])-relax_margin_u >= 0 else 0
        v_max = max(uv_pt[1])+relax_margin_v if max(uv_pt[1])+relax_margin_v <=img_size[1] else img_size[1]
        v_min = min(uv_pt[1])-relax_margin_v if min(uv_pt[1])-relax_margin_v >= 0 else 0

        return [u_min, v_min,  u_max, v_max]

## Original Image and Annotations

In [ ]:
img_id=10
n_kps = 12
img_path = f'/content/drive/My Drive/Envisat/data/Envisat_Set1/traj_test/test_traj_150m/0-999/image_{img_id}.jpg'
img = Image.open(img_path,'r')
gray = cv2.cvtColor(np.asarray(img),  cv2.COLOR_BGR2GRAY)
img1 = np.zeros((gray.shape[0],gray.shape[1],3),dtype='uint8')
img1[:,:,0] = gray 
img1[:,:,1] = gray 
img1[:,:,2] = gray 
fig = plt.figure(figsize=(10,10))
ax = fig.gca()
ax.imshow(img1)
img= Image.fromarray(img1)


img_name = (img_path.split('/')[-1]).split('.')[0]
cocofile = '/content/drive/My Drive/Envisat/data/Envisat_Set1/traj_test/test_traj_12pts_150m.json'
coco = COCO(cocofile)
annIds = coco.getAnnIds(imgIds = img_id)
annotations = coco.loadAnns(annIds)
bbox= {img_name: annotations[0]['bbox']}
kps= {img_name: annotations[0]['keypoints']}
coords = kps[img_name] 
kps_gt = np.zeros((n_kps,2))
uv_pt = ([],[])
for k in range(n_kps):
  x = coords[3*k]
  y =coords[3*k+1]
  kps_gt[k,:] = [x,y]
  uv_pt[0].append(x)
  uv_pt[1].append(y)

xyxy = compute_bbox((512,512),uv_pt) 
xywh = [ xyxy[0],xyxy[1], xyxy[2]-xyxy[0], xyxy[3]-xyxy[1]] 
bbox = {img_name: xywh} if n_kps==12 else bbox

In [ ]:
plt.imshow(img)

Image Transforms

In [ ]:
jitter = vision.transforms.ColorJitter(brightness= 0.4,contrast=0.4)#, saturation = 0.1, hue = 0.5)
jitter = jitter(img)
plt.imshow(jitter)

Fake args Namespace Class

In [ ]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)


# Pose Inference: IPython

In [ ]:
'''DO NOT JUDGE THIS CODE. 
   IT WAS WRITTEN AT 3 AM 
   DURING COVID-19 QUARANTINE
'''

from demo.sat_pose_inference import main as infer_kepyoints

cfg_path = '/content/drive/My Drive/Envisat/HRNet_regression/experiments/EXP/EXP5_w32_256x256_adam_lr1e-3_ENVISAT+IC_imagenet.yaml'
#cfg_path = '/content/drive/My Drive/Envisat/HRNet_regression/experiments/coco/hrnet/w32_256x256_adam_lr1e-3_ENVISAT-1_test.yaml'

model_path = 'output/Envisat+IC/envisat_coco_envisat/pose_hrnet/EXP5_w32_256x256_adam_lr1e-3_ENVISAT+IC_imagenet/model_best.pth'
#model_path = 'output/Envisat_Set1/coco/pose_hrnet/w32_256x256_adam_lr1e-3_ENVISAT-1/model_best.pth'


''' Name you test data_set here. This helps segment tests in /output/sat_pose_inferences by folder of data_name'''
data_name = 'beta_test'

img_bgr = np.array(jitter)
img_in0 =img_bgr[:, :, [2, 1, 0]]
img_in1 = img_in0.copy();
img_in = cv2.GaussianBlur(img_in1,(1,1),0)
bbox_in = bbox


args = Namespace(
    cfg= cfg_path,
    data_name = data_name,
    logDir = 'log/',
    outputDir = 'output/', 
    modelDir = '',
    dataDir = '',
    prevModelDir = '',
    writeBoxFrames = False,
    opts = ['TEST.MODEL_FILE', model_path],)



#str_dir = f'/content/drive/My\ Drive/Envisat/data/Envisat_Set1/traj_test/test_traj_150m/0-999/image_{img_id}.jpg'
kps_pred, maxvals = infer_kepyoints(image= img_in, bbox= bbox_in, args = args)
kps_pred

Show Result

In [ ]:
bbox_in

In [ ]:
error = kps_gt-kps_pred[0,:]
dr_error = np.sqrt( error[:,0]**2 + error[:,1]**2 )
mean_dr = np.mean(dr_error)
median_dr = np.median(dr_error)
print(f" Mean Pixel Error: {mean_dr} px \n Median Pixel Error: {median_dr} px \n Max Pixel Error: {np.max(dr_error)} px in keypoint number {np.argmax(dr_error)}")

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(img_in)

In [ ]:

output = f'output/sat_pose_inferences/envisat_coco_envisat/beta_test/poses/image_{img_id}_pose.jpg'
#output = f'output/sat_pose_inferences/envisat_coco/beta_test/poses/image_{img_id}_pose.jpg'
box = [int(coord) for coord in bbox[list(bbox)[0]]]
img_out = Image.open(output)

x1 = box [0] 
y1 = box[1]
w = box[2]
h = box[3]
x2= box[2]+box[0]
y2 = box[3]+box[1]

cv2.rectangle(np.array(img),(x1,y1),(x2,y2), color = (0,255,0))

img_crop = img_out.crop((x1,y1,x2,y2))
fig = plt.figure(figsize=(10,10))
ax = fig.gca()

#ax.imshow(np.asarray(img_crop))

ax.imshow(img_out)

# New Section

In [ ]:
fig1 = plt.figure(figsize=(10,10))
ax = fig.gca()
ax2.imshow(img_out)

# Pose Inference: CLI

In [ ]:
'''DO NOT JUDGE THIS CODE. 
   IT WAS WRITTEN AT 3 AM 
   DURING COVID-19 QUARANTINE
'''


str_dir = f'/content/drive/My\ Drive/Envisat/data/Envisat_Set1/traj_test/test_traj_150m/0-999/image_{img_id}.jpg'
with torch.cuda.device(0):
  !python demo/sat_pose_inference_cmd.py --cfg experiments/coco/hrnet/w32_512x512_adam_lr1e-3_ENVISAT-1_traj_test.yaml \
      --data_name beta_test \
      --data_type image \
      --data_path $str_dir \
      --bbox demo/temp.json \
      TEST.MODEL_FILE output/Envisat_1/coco/pose_hrnet/w32_256x256_adam_lr1e-3_ENVISAT-1/model_best.pth \


output = f'output/sat_pose_inferences/coco/beta_test/poses/image_{img_id}_pose.jpg'
box = bbox[list(bbox)[0]]
img = Image.open(output)

bbox = (box[0],box[1],box[2]+box[0],box[3]+box[1])
cv2.rectangle(np.array(img),(bbox[0],bbox[1], bbox[2], bbox[3]))

img_crop = img.crop(bbox)
fig = plt.figure(figsize=(10,10))
ax = fig.gca()
ax.imshow(img)
#ax.imshow(np.asarray(img_crop))
fig1 = plt.figure(figsize=(10,10))
ax1 = fig.gca()
ax1.imshow(img)
ax1.imshow(img_crop)

In [ ]:
os.getcwd()

In [ ]:
img

In [ ]:
print(bbox)